created by Finn Buchrieser

In [1]:
# importing all packages
import pandas as pd
import re
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from stop_words import get_stop_words
from nltk import word_tokenize
from tqdm import tqdm, tqdm_pandas
from multiprocessing.pool import Pool
import time
tqdm.pandas(mininterval = 3)
import string

In [2]:
# reading all datasets from csv files
# change encoding dependent on file source (R or Python) or check encoding of file
df_che = pd.read_csv(r"E:\Organization_Project\files\Buchrieser_Meta_Dataset\che_dataset.csv", encoding = "ANSI")
df_inf = pd.read_csv(r"E:\Organization_Project\files\Buchrieser_Meta_Dataset\inf_dataset.csv", encoding = "ANSI")
df_hum = pd.read_csv(r"E:\Organization_Project\files\Buchrieser_Meta_Dataset\hum_dataset.csv", encoding = "ANSI")
df_mat = pd.read_csv(r"E:\Organization_Project\files\Buchrieser_Meta_Dataset\math_dataset.csv", encoding = "ANSI")
df_bio = pd.read_csv(r"E:\Organization_Project\files\Edlinger_Meta_Dataset\df_bio.csv", encoding = "ANSI")
df_phy = pd.read_csv(r"E:\Organization_Project\files\Edlinger_Meta_Dataset\df_phy.csv", encoding = "ANSI")
df_psy = pd.read_csv(r"E:\Organization_Project\files\Edlinger_Meta_Dataset\df_psy.csv", encoding = "ANSI")

In [3]:
df_mat.head()

,Index,Date,Authors,ID,Journal,Mesh,Subject,abstract
0,1.0,2022 Jun 6,Dudchenko A,35673175.0,Studies in health technology and informatics,Data Analysis[Mesh Terms],math,Abstract ...
1,2.0,2022,Zhao J,35669742.0,Frontiers in public health,Data Analysis[Mesh Terms],math,Abstract ...
2,3.0,2022 Jun 6,Liu T,35668363.0,BMC bioinformatics,Data Analysis[Mesh Terms],math,Abstract ...
3,4.0,2022 Jun 6,Qi P,35659217.0,BMC medical informatics and decision making,Data Analysis[Mesh Terms],math,Abstract ...
4,5.0,2022,Jeong K,35657592.0,"Methods in molecular biology (Clifton, N.J.)",Data Analysis[Mesh Terms],math,Abstract ...


In [10]:
df_psy.head()

,DATE,AUTHOR,ID,JOURNAL,KEYWORD,TOPIC,abstract
0,2021 Dec,Hihara S,34928673,Developmental psychology,Adolescent Psychology[Mesh Terms],psy,Abstract ...
1,2021 Nov 16,Zhang Q,34831792,International journal of environmental researc...,Adolescent Psychology[Mesh Terms],psy,Abstract ...
2,2021 Dec,Weavers B,34672991,The lancet. Psychiatry,Adolescent Psychology[Mesh Terms],psy,Abstract ...
3,2021,Rousseau E,34648589,PloS one,Adolescent Psychology[Mesh Terms],psy,Abstract ...
4,2021 Jul-Aug,Prinstein MJ,34613846,Journal of clinical child and adolescent psych...,Adolescent Psychology[Mesh Terms],psy,Abstract ...


In [3]:
#df_bio.drop(df_bio.columns[0], axis = 1, inplace = True)
#df_phy.drop(df_phy.columns[0], axis = 1, inplace = True)
#df_psy.drop(df_psy.columns[0], axis = 1, inplace = True)
df_che.drop(df_che.columns[0], axis = 1, inplace = True)
df_inf.drop(df_inf.columns[0], axis = 1, inplace = True)
df_hum.drop(df_hum.columns[0], axis = 1, inplace = True)
df_mat.drop(df_mat.columns[0], axis = 1, inplace = True)

In [4]:
df_che.rename(columns = {"Mesh": "KEYWORD", "Journal": "JOURNAL", "Authors" : "AUTHOR", "Date" : "DATE", "Subject": "SCIENCE", "abstract" : "Abstract"}, inplace = True)
df_inf.rename(columns = {"Mesh": "KEYWORD", "Journal": "JOURNAL", "Authors" : "AUTHOR", "Date" : "DATE", "Subject": "SCIENCE", "abstract" : "Abstract"}, inplace = True)
df_hum.rename(columns = {"Mesh": "KEYWORD", "Journal": "JOURNAL", "Authors" : "AUTHOR", "Date" : "DATE", "Subject": "SCIENCE", "abstract" : "Abstract"}, inplace = True)
df_mat.rename(columns = {"Mesh": "KEYWORD", "Journal": "JOURNAL", "Authors" : "AUTHOR", "Date" : "DATE", "Subject": "SCIENCE", "abstract" : "Abstract"}, inplace = True)
df_bio.rename(columns = {"TOPIC": "SCIENCE", "abstract" : "Abstract"}, inplace = True)
df_phy.rename(columns = {"TOPIC": "SCIENCE", "abstract" : "Abstract"}, inplace = True)
df_psy.rename(columns = {"TOPIC": "SCIENCE", "abstract" : "Abstract"}, inplace = True)

In [5]:
df_che["SCIENCE"] = df_che.SCIENCE.apply(lambda x: str(x)[0:3])
df_inf["SCIENCE"] = df_inf.SCIENCE.apply(lambda x: str(x)[0:3])
df_hum["SCIENCE"] = df_hum.SCIENCE.apply(lambda x: str(x)[0:3])
df_mat["SCIENCE"] = df_mat.SCIENCE.apply(lambda x: str(x)[0:3])

In [6]:
df_combined = pd.concat([df_che, df_inf, df_hum, df_mat, df_bio, df_phy, df_psy], ignore_index = True)
df_combined.name = 'Combined dataframe'

In [7]:
# changing dtype of columns to string
df_combined["SCIENCE"] = df_combined["SCIENCE"].astype(str)
df_combined["KEYWORD"] = df_combined["KEYWORD"].astype(str)
df_combined["Abstract"] = df_combined["Abstract"].astype(str)

In [11]:
# data exploration

df_combined.shape

(4683280, 7)

In [8]:
# Preprocessing
# creating a dictionary for written date to numerical date conversion
dict_of_str_dates = {"Jan" : "01", "Feb" : "02", "Mar" : "03", "Apr" : "04",
        	         "May" : "05", "Jun" : "06", "Jul" : "07", "Aug" : "08",
                     "Sep" : "09", "Oct" : "10", "Nov" : "11", "Dez" : "12",
                     "1st Quart" : "01", "2nd Quart": "04", "3rd Quart": "07", "En" : "01", "De" : "12",
                       "Oc" : "10", "Au" : "08", "Se" : "09", "Ju" : "07", "Ap" : "04", "Ma" : "03",
                         "Fe" : "02", "Ja" : "01", "N" : "11", "D" : "12", "O" : "10",
                           "A" : "08", "S" : "09", "J" : "07",
                       "M" : "03", "F" : "02", "Jn" : "06", "Jl" : "07", "Ag" : "08",
                         "Spt" : "09", "Oc" : "10", "Nv" : "11", "Dc" : "12",
                           "1st Quart" : "01", "2nd Quart" : "04", "3rd Quart" : "07",
                     "4th Quart" : "10"}

def preprocessing(df):
    temp_len_1 = len(df.index)
    print(f"Unprocessed Dataframe has {temp_len_1} Rows\n")
    # change column to correct variable type
    df["Abstract"] = df["Abstract"].astype(str)
    # removign mutilingual abstracts
    df.drop(df[df["Abstract"].apply(lambda x: bool(re.search("Abstract in", x))) == True].index, inplace = True)
    # seperating keywords from abstracts
    df["Abstract"] = df["Abstract"].apply(lambda x: re.split(r"Keywords:", x)[0])
    # removing the word abstract from start of abstracts
    df["Abstract"] = df["Abstract"].apply(lambda x: re.sub('Abstract','', x))
    # drop all rows with missing values
    df.dropna(inplace = True)
    # removing all duplicate abstracts
    df.drop_duplicates(subset=['ID'], inplace=True)
    temp_len_2 = len(df.index)
    removed_1 = temp_len_1-temp_len_2
    print(f"Splitted keywords and removed multilingual abstracts/droped NANs/ removed duplicates from Combined Dataframe"
        f"\n {removed_1} rows were removed")
    
    # casting all dates to string
    df["DATE"] = df["DATE"].astype(str)
    # remove white spaces in front of abstract/date and trailing white spaces
    df["Abstract"] = df["Abstract"].apply(lambda x: x.strip())
    df["DATE"] = df["DATE"].apply(lambda x: x.strip())
    # remove tabs and multible white spaces
    df["Abstract"] = df["Abstract"].apply(lambda x: re.sub(' +', ' ', x))

    print(f"Removed whitespaces and tabs from Combined Dataframe")

    # remove punctuation from date
    df["DATE"] = df["DATE"].apply(lambda x: re.sub('\\.','', x))
    # change dates like Mar-Apr and 06-07 to only the first month of time period.
    df["DATE"] = df["DATE"].apply(lambda x: re.split(r"-|/", x)[0])

    print(f"Removed punctuation from date and changed date ranges to single date Combined Dataframe\n")

    # remove abstracts that are shorter than 30 characters
    df.drop(df[df["Abstract"].apply(lambda x: len(x)) < 30].index, inplace = True)
    # remove abstracts with less than 50 full words
    df.drop(df[df["Abstract"].apply(lambda x: len(x.split())) < 50].index, inplace = True)
    # remove abstracts with more than 500 full words
    df.drop(df[df["Abstract"].apply(lambda x: len(x.split())) > 500].index, inplace = True)

    temp_len_1 = len(df.index)
    removed_2 = temp_len_2-temp_len_1
    print(f"Removed to short and too long abstracts from Combined Dataframe"
        f"\n {removed_2} rows were removed\n")


    # only keeping the year in case of weird date format
    df["DATE"] = df["DATE"].apply(lambda x: x[0:4] if len(x)>10 else x)
    # replace season names with month in dates
    df["DATE"] = df["DATE"].apply(lambda x: re.sub('(?i)Fall','10', x))
    df["DATE"] = df["DATE"].apply(lambda x: re.sub('(?i)Winter|Win','01', x))
    df["DATE"] = df["DATE"].apply(lambda x: re.sub('(?i)Summer|Sum','07', x))
    df["DATE"] = df["DATE"].apply(lambda x: re.sub('(?i)Spring','04', x))
    df["DATE"] = df["DATE"].apply(lambda x: re.sub('(?i)Christmas','12', x))
    # replacing written dates with numerical dates
    for pair in dict_of_str_dates:
        df["DATE"] = df["DATE"].apply(lambda x: re.sub(f'(?i){pair}',dict_of_str_dates[pair], x))
    # switches month and year if in incorrect order
    df["DATE"] = df["DATE"].apply(lambda x: x if bool(re.search(r'(19[4-9][0-9]|20[0-2][0-9])', x[0:4])) == True else (f'{x[3:8]} {x[0:2]}'))
    # drop rows in which date is too short
    df.drop(df[df["DATE"].apply(lambda x: len(x)) < 4].index, inplace = True)
  
    # removing years older than 1980
    df.drop(df[df["DATE"].apply(lambda x: bool(re.search("190[0-9]|19[1-7][0-9]", x))) == True].index, inplace = True)
    # adding default day/month to date
    df["DATE"] = df["DATE"].apply(lambda x: re.sub(" ","-",f'{x[0:7]} 01') if len(x) > 4 else re.sub(" ","-",f'{x} 01 01'))
    # removing all abstracts that still dont fit into the datetime class
    df.drop(df[df["DATE"].apply(lambda x: bool(re.search(r'(19[4-9][0-9]|20[0-2][0-9])-[0-1][0-9]-[0-1][0-9]', x)) == False)].index, inplace = True)
    # remove impossible month/dates
    df["DATE"] = df["DATE"].apply(lambda x: x[0:4] if int(x[5]) > 1 else x)
    # finally converting all dates to np.datetime format for further analysis
    
    df["DATE"] = pd.to_datetime(df["DATE"], format="%Y-%m-%d")

    ############################ this line does not work properly ############################
    # names_to_keep =  ['che', 'inf', 'mat', 'psy', 'bio', 'hum', 'phy']
    # df = df[df['SCIENCE'].isin(names_to_keep)]

    temp_len_2 = len(df.index)
    removed_3 = temp_len_1-temp_len_2
    print(f"Converted all dates in Combined Dataframe into Datetime format"
        f"\n {removed_3} rows were removed\n")

    # removing stopwords  and punctuation from data
    # stop_words = get_stop_words('en')
    # df['Abstract'] = df['Abstract'].progress_apply(lambda x: ' '.join([word for word in word_tokenize(x.lower()) if word not in (stop_words)]))
    # df.Abstract = df.Abstract.progress_apply(lambda x: ' '.join(word.strip(string.punctuation) for word in x.split()))
    # print(f"Removed Stopwords from {str(df.name)}")
    # print(f"{str(df.name)} fully preprocessed\n")
    removed_sum = removed_1 + removed_2 + removed_3
    print(f"{removed_sum} rows were removed from dataframe in total\n")

In [9]:
preprocessing(df_combined)

Unprocessed Dataframe has 4683280 Rows

Splitted keywords and removed multilingual abstracts/droped NANs/ removed duplicates from Combined Dataframe
 719196 rows were removed
Removed whitespaces and tabs from Combined Dataframe
Removed punctuation from date and changed date ranges to single date Combined Dataframe

Removed to short and too long abstracts from Combined Dataframe
 667623 rows were removed

Converted all dates in Combined Dataframe into Datetime format
 55560 rows were removed

1442379 rows were removed from dataframe in total



In [12]:
names_to_keep =  ['che', 'inf', 'mat', 'psy', 'bio', 'hum', 'phy']
df_combined = df_combined[df_combined['SCIENCE'].isin(names_to_keep)]

In [15]:
df_combined.to_csv(r"E:\Organization_Project\files\raw_abtracts_combined.csv", index=False)